In [1]:
!pip install -qq scikit-learn==1.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 97.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class CFG:
    train_path: Path = Path("/kaggle/input/playground-series-s5e4/train.csv")
    test_path: Path = Path("/kaggle/input/playground-series-s5e4/test.csv")
    sub_path: Path = Path("/kaggle/input/playground-series-s5e4/sample_submission.csv")
    pltpd_path: Path = Path("/kaggle/input/podcast-listening-time-prediction-dataset/podcast_dataset.csv")
    
    num_fold: int = 5
    dev_mode: bool = False

    # Model parameters
    n_iter: int = 10000
    max_depth: int = -1
    num_leaves: int = 1024
    colsample_bytree: float = 0.7
    learning_rate: float = 0.04

    objective: str = "l2"
    metric: str = "rmse"
    verbosity: int = -1

    random_state: int = 42
    shuffle: bool = True
    encoded_columns_start: int = -91
    log_eval: int = 100
    early_stopping: int = 200

    # debug = True


cfg = CFG()


In [3]:
import gc
from itertools import combinations

import numpy as np
import polars as pl
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def calc_rmse(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse


re_dict = {}
re_dict["podc_dict"] = {
    "Mystery Matters": 0,
    "Joke Junction": 1,
    "Study Sessions": 2,
    "Digital Digest": 3,
    "Mind & Body": 4,
    "Fitness First": 5,
    "Criminal Minds": 6,
    "News Roundup": 7,
    "Daily Digest": 8,
    "Music Matters": 9,
    "Sports Central": 10,
    "Melody Mix": 11,
    "Game Day": 12,
    "Gadget Geek": 13,
    "Global News": 14,
    "Tech Talks": 15,
    "Sport Spot": 16,
    "Funny Folks": 17,
    "Sports Weekly": 18,
    "Business Briefs": 19,
    "Tech Trends": 20,
    "Innovators": 21,
    "Health Hour": 22,
    "Comedy Corner": 23,
    "Sound Waves": 24,
    "Brain Boost": 25,
    "Athlete's Arena": 26,
    "Wellness Wave": 27,
    "Style Guide": 28,
    "World Watch": 29,
    "Humor Hub": 30,
    "Money Matters": 31,
    "Healthy Living": 32,
    "Home & Living": 33,
    "Educational Nuggets": 34,
    "Market Masters": 35,
    "Learning Lab": 36,
    "Lifestyle Lounge": 37,
    "Crime Chronicles": 38,
    "Detective Diaries": 39,
    "Life Lessons": 40,
    "Current Affairs": 41,
    "Finance Focus": 42,
    "Laugh Line": 43,
    "True Crime Stories": 44,
    "Business Insights": 45,
    "Fashion Forward": 46,
    "Tune Time": 47,
}
re_dict["genr_dict"] = {
    "True Crime": 0,
    "Comedy": 1,
    "Education": 2,
    "Technology": 3,
    "Health": 4,
    "News": 5,
    "Music": 6,
    "Sports": 7,
    "Business": 8,
    "Lifestyle": 9,
}
re_dict["week_dict"] = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3,
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6,
}
re_dict["time_dict"] = {"Morning": 10, "Afternoon": 14, "Evening": 17, "Night": 21}
re_dict["sent_dict"] = {"Negative": 0, "Neutral": 1, "Positive": 2}


pl_i_type = pl.Int32
pl_f_type = pl.Float32


def cast_numeric_dtypes(df):
    float_cols = [col for col in df.columns if df.schema[col] == pl.Float64 or df.schema[col] == pl.Float32]
    int_cols = [col for col in df.columns if df.schema[col] == pl.Int64 or df.schema[col] == pl.Int32]

    if float_cols:
        df = df.with_columns([pl.col(col).cast(pl_f_type) for col in float_cols])
    if int_cols:
        df = df.with_columns([pl.col(col).cast(pl_i_type) for col in int_cols])

    return df


def preprocess(df, df_train=None):
    df = cast_numeric_dtypes(df)
    df = df.with_columns(pl.col("Episode_Title").str.slice(8).cast(pl.Int32).alias("Episode_Num")).drop("Episode_Title")

    # Convert categorical variables using mapping
    for col, mapping in [
        ("Genre", re_dict["genr_dict"]),
        ("Podcast_Name", re_dict["podc_dict"]),
        ("Publication_Day", re_dict["week_dict"]),
        ("Publication_Time", re_dict["time_dict"]),
        ("Episode_Sentiment", re_dict["sent_dict"]),
    ]:
        df = df.with_columns(pl.col(col).replace(mapping).alias(col))

    # Cap extreme values
    df = df.with_columns(
        pl.when(pl.col("Episode_Length_minutes") > 121.0).then(121.0).otherwise(pl.col("Episode_Length_minutes")).alias("Episode_Length_minutes"),
        pl.when(pl.col("Number_of_Ads") > 103.91).then(103.91).otherwise(pl.col("Number_of_Ads")).alias("Number_of_Ads"),
    )

    # Create NaN indicator columns
    df = df.with_columns(
        pl.col("Episode_Length_minutes").is_null().cast(pl.Utf8).cast(pl.Categorical).alias("Episode_Length_minutes_NaN"),
        pl.col("Guest_Popularity_percentage").is_null().cast(pl.Utf8).cast(pl.Categorical).alias("Guest_Popularity_percentage_NaN"),
        pl.col("Number_of_Ads").is_null().cast(pl.Utf8).cast(pl.Categorical).alias("Number_of_Ads_NaN"),
    )

    # Fill NA values with median
    if df_train is None:
        df_train = df.clone()

    e_median = df_train.select(pl.col("Episode_Length_minutes").median()).item()
    g_median = df_train.select(pl.col("Guest_Popularity_percentage").median()).item()
    n_median = df_train.select(pl.col("Number_of_Ads").median()).item()

    df = df.with_columns(
        pl.col("Episode_Length_minutes").fill_null(e_median),
        pl.col("Guest_Popularity_percentage").fill_null(g_median),
        pl.col("Number_of_Ads").fill_null(n_median),
    )

    return df


def feature_eng(df, df_train):
    # Cyclical features for day and time
    df = df.with_columns(
        # Day features
        pl.col("Publication_Day").cast(pl_f_type).mul(2 * np.pi / 7).sin().alias("Day_sin"),
        pl.col("Publication_Day").cast(pl_f_type).mul(2 * np.pi / 7).cos().alias("Day_cos"),
        pl.col("Publication_Day").cast(pl_f_type).mul(4 * np.pi / 7).sin().alias("Day_sin2"),
        pl.col("Publication_Day").cast(pl_f_type).mul(4 * np.pi / 7).cos().alias("Day_cos2"),
        # Time features
        pl.col("Publication_Time").cast(pl_f_type).mul(2 * np.pi / 4).sin().alias("Time_sin"),
        pl.col("Publication_Time").cast(pl_f_type).mul(2 * np.pi / 4).cos().alias("Time_cos"),
        pl.col("Publication_Time").cast(pl_f_type).mul(4 * np.pi / 24).sin().alias("Time_sin2"),
        pl.col("Publication_Time").cast(pl_f_type).mul(4 * np.pi / 24).cos().alias("Time_cos2"),
        # Ratio features
        (pl.col("Episode_Length_minutes") / (pl.col("Number_of_Ads") + 1)).fill_null(0).alias("Length_per_Ads"),
        (pl.col("Episode_Length_minutes") / (pl.col("Host_Popularity_percentage") + 1)).fill_null(0).alias("Length_per_Host"),
        (pl.col("Episode_Length_minutes") / (pl.col("Guest_Popularity_percentage") + 1)).fill_null(0).alias("Length_per_Guest"),
        # Episode length features
        pl.col("Episode_Length_minutes").floor().alias("ELen_Int"),
        (pl.col("Episode_Length_minutes") - pl.col("Episode_Length_minutes").floor()).alias("ELen_Dec"),
        pl.col("Host_Popularity_percentage").floor().alias("HPperc_Int"),
        (pl.col("Host_Popularity_percentage") - pl.col("Host_Popularity_percentage").floor()).alias("HPperc_Dec"),
        # Sentiment features
        (pl.col("Episode_Sentiment") == "2").cast(pl.Int8).alias("Is_Positive_Sentiment"),
        pl.when(pl.col("Episode_Sentiment") == "2").then(0.75).otherwise(0.717).cast(pl_f_type).alias("Sentiment_Multiplier"),
        # Squared features
        (pl.col("Episode_Length_minutes") ** 2).alias("Episode_Length_squared"),
        (pl.col("Episode_Length_minutes") ** 3).alias("Episode_Length_squared2"),
    )

    # Add expected listening time based on sentiment
    df = df.with_columns((pl.col("Episode_Length_minutes") * pl.col("Sentiment_Multiplier")).alias("Expected_Listening_Time_Sentiment"))

    # Convert columns to categorical
    for col in ["Podcast_Name", "Genre", "Publication_Day", "Publication_Time", "Episode_Sentiment", "Episode_Num"]:
        df = df.with_columns(pl.col(col).cast(pl.Utf8).cast(pl.Categorical))

    return df


def cols_encode(df):
    # Order of importance
    columns_to_encode = [
        "Host_Popularity_percentage",
        "Guest_Popularity_percentage",
        "Episode_Length_minutes",
        "Episode_Num",
        "Podcast_Name",
        "Publication_Day",
        "Publication_Time",
        # "Episode_Sentiment",
        "Genre",
        "Number_of_Ads",
        "Episode_Length_minutes_NaN",
        "Guest_Popularity_percentage_NaN",
        "HPperc_Int",
        "HPperc_Dec",
        "ELen_Int",
        "ELen_Dec",
        "Length_per_Ads",
    ]

    pair_size = [2]

    # Pure polars implementation for combinations
    for r in pair_size:
        combinations_list = list(combinations(columns_to_encode, r))
        batch_size = 20

        print("\n pair_size:", r, "\n")

        for i in range(0, len(combinations_list), batch_size):
            batch = combinations_list[i : i + batch_size]

            for cols in tqdm(batch):
                new_col_name = "colen_" + "_".join(cols)
                concat_expr = pl.col(cols[0]).cast(pl.Utf8)

                for col_name in cols[1:]:
                    concat_expr = concat_expr + "_" + pl.col(col_name).cast(pl.Utf8)

                df = df.with_columns(concat_expr.alias(new_col_name).cast(pl.Categorical))

            gc.collect()

            mem_usage = sum(df.estimated_size() for col in df.columns) / (1024 * 1024)
            print(f"Memory usage: {mem_usage:.2f} MB")
            print(f"Total number of columns: {len(df.columns)}")

        print("=" * 20)

    columns_to_encode = [
        "Host_Popularity_percentage",
        "Guest_Popularity_percentage",
        "Episode_Length_minutes",
        # "Episode_Num",
        "Podcast_Name",
        "Publication_Day",
        # "Publication_Time",
        "Episode_Length_minutes_NaN",
        # "Guest_Popularity_percentage_NaN",
        # "HPperc_Int",
        # "HPperc_Dec",
        "ELen_Int",
        "ELen_Dec",
        "Length_per_Ads",
    ]

    pair_size = [3]

    # Pure polars implementation for combinations
    for r in pair_size:
        combinations_list = list(combinations(columns_to_encode, r))
        batch_size = 20

        print("\n pair_size:", r, "\n")

        for i in range(0, len(combinations_list), batch_size):
            batch = combinations_list[i : i + batch_size]

            for cols in tqdm(batch):
                new_col_name = "colen_" + "_".join(cols)
                concat_expr = pl.col(cols[0]).cast(pl.Utf8)

                for col_name in cols[1:]:
                    concat_expr = concat_expr + "_" + pl.col(col_name).cast(pl.Utf8)

                df = df.with_columns(concat_expr.alias(new_col_name).cast(pl.Categorical))

            gc.collect()

            mem_usage = sum(df.estimated_size() for col in df.columns) / (1024 * 1024)
            print(f"Memory usage: {mem_usage:.2f} MB")
            print(f"Total number of columns: {len(df.columns)}")

        print("=" * 20)

    return df



In [4]:
import gc
import numpy as np
import polars as pl
from sklearn.model_selection import KFold
from sklearn.preprocessing import TargetEncoder
import lightgbm as lgb
import wandb
from kaggle_secrets import UserSecretsClient
from dataclasses import asdict


# Custom wandb callback
class WandbCallback:
    def __init__(self, log_every=50, log_feature_importance=True):
        self.log_every = log_every
        self.iteration = 0
        self.log_feature_importance = log_feature_importance

    def __call__(self, env):
        if self.iteration % self.log_every == 0:
            metrics = {}
            for dataset_name, eval_name, value, _ in env.evaluation_result_list:
                metric_name = f"{dataset_name}/{eval_name}"
                metrics[metric_name] = value

            # Log feature importance if requested
            if self.log_feature_importance and hasattr(env, "model"):
                feature_names = env.model.feature_name()
                importance = env.model.feature_importance(importance_type="split")
                feature_importance = {name: imp for name, imp in zip(feature_names, importance)}

                # Log to wandb
                wandb.log(
                    {
                        "feature_importance": wandb.Table(
                            data=[[k, v] for k, v in sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)],
                            columns=["feature", "importance"],
                        )
                    },
                    step=self.iteration,
                )

                # Log top 10 features
                wandb.log(
                    {
                        "feature_importance_plot": wandb.plot.bar(
                            wandb.Table(
                                data=[[k, v] for k, v in sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:10]],
                                columns=["feature", "importance"],
                            ),
                            "feature",
                            "importance",
                            title="Top Feature Importance",
                        )
                    },
                    step=self.iteration,
                )

            wandb.log(metrics, step=self.iteration)

        self.iteration += 1
        return False
        
def get_dfs_with_kfold(cfg):
    # Read CSV files
    df_train = pl.read_csv(cfg.train_path)
    df_test = pl.read_csv(cfg.test_path)
    
    test_ids = df_test["id"]

    df_train = df_train.drop("id")
    df_test = df_test.drop("id")
    
    target_col = "Listening_Time_minutes"
    
    # Load and prepare podcast data
    df_pltpd = pl.read_csv(cfg.pltpd_path)
    df_pltpd = df_pltpd.filter(pl.col("Listening_Time_minutes").is_not_null())
    y_pltpd = df_pltpd["Listening_Time_minutes"]
    df_pltpd = df_pltpd.drop("Listening_Time_minutes")
    df_pltpd = df_pltpd.with_columns(pl.col("Number_of_Ads").cast(pl.Float64))
    
    # Create KFold
    kf = KFold(n_splits=cfg.num_fold, random_state=42, shuffle=True)
    
    # Get all row indices
    all_indices = np.arange(len(df_train))
    
    # Initialize predictions array for test data
    test_predictions = np.zeros(len(df_test))
    
    # Train models with cross-validation
    for fold, (idx_train, idx_valid) in enumerate(kf.split(all_indices)):
        print(f"Training fold {fold+1}/{cfg.num_fold}")

        # Use direct indexing to select rows from the Polars DataFrame
        X_train = df_train.drop(target_col)[idx_train.tolist()]
        y_train = df_train.select(target_col)[idx_train.tolist()]
        X_valid = df_train.drop(target_col)[idx_valid.tolist()]
        y_valid = df_train.select(target_col)[idx_valid.tolist()]
        
        # Concatenate with podcast data
        X_train = pl.concat([X_train, df_pltpd], how="vertical")
        y_train = pl.concat([y_train, pl.DataFrame({target_col: y_pltpd})], how="vertical")

        # Only use this line for debugging with a small sample
        if hasattr(cfg, 'debug') and cfg.debug:
            X_train = X_train.sample(100)
            y_train = y_train.sample(100)

        # Preprocess
        X_train = preprocess(X_train)
        X_valid = preprocess(X_valid, X_train)
        
        # Feature engineering
        df_train_with_target = X_train.with_columns(y_train.select(pl.col(target_col)).rename({target_col: target_col}))
        X_train = feature_eng(X_train, df_train_with_target)
        X_valid = feature_eng(X_valid, df_train_with_target)
        
        # Encoding
        before_encode_len = len(X_train.columns)
        X_train = cols_encode(X_train)
        X_valid = cols_encode(X_valid)
        
        # Target encoding - we need to apply this with Polars
        encoded_columns = X_train.columns[before_encode_len:]
        
        # For target encoding, we'll use a custom function that works with polars
        X_train, X_valid, encoder = target_encode_polars(
            X_train, y_train, X_valid, 
            columns=encoded_columns,
            random_state=cfg.random_state
        )

        X_train = cast_numeric_dtypes(X_train)
        X_valid = cast_numeric_dtypes(X_valid)
        
        # Convert to numpy arrays for LightGBM
        X_train_arr = X_train.to_pandas()
        y_train_arr = y_train.to_pandas()
        X_valid_arr = X_valid.to_pandas()
        y_valid_arr = y_valid.to_pandas()
        
        # Train model
        model = lgb.LGBMRegressor(
            n_iter=cfg.n_iter,
            max_depth=cfg.max_depth,
            num_leaves=cfg.num_leaves,
            colsample_bytree=cfg.colsample_bytree,
            learning_rate=cfg.learning_rate,
            objective=cfg.objective,
            metric=cfg.metric,
            verbosity=cfg.verbosity,
            random_state=42,
        )
        
        model.fit(
            X_train_arr, y_train_arr,
            eval_set=[(X_train_arr, y_train_arr), (X_valid_arr, y_valid_arr)],
            callbacks=[
                lgb.log_evaluation(cfg.log_eval),
                lgb.early_stopping(cfg.early_stopping),
                WandbCallback(log_every=50),
            ],
            feature_name=X_train.columns,
        )
        
        # Process test data
        X_test = preprocess(df_test, X_train)
        X_test = feature_eng(X_test, df_train_with_target)
        X_test = cols_encode(X_test)
        X_test = apply_target_encoding(X_test, encoder, encoded_columns)
        X_test = cast_numeric_dtypes(X_test)
        X_test_arr = X_test.to_pandas()
        
        # Add predictions
        fold_pred = model.predict(X_test_arr)
        test_predictions += fold_pred
        
        # Log fold performance
        val_score = model.best_score_["valid_1"][cfg.metric]
        print(f"Fold {fold+1} validation score: {val_score}")
        wandb.log({f"fold_{fold+1}_val_score": val_score})
        
        # Cleanup
        gc.collect()
    
    return test_predictions


def target_encode_polars(X_train, y_train, X_valid, columns, random_state=42):

    # Initialize the encoder
    encoder = TargetEncoder(random_state=random_state)
    
    # We'll need numpy arrays for the encoding
    X_train_numpy = X_train.select(columns).to_numpy()
    y_train_numpy = y_train.to_numpy()
    X_valid_numpy = X_valid.select(columns).to_numpy()
    
    # Fit and transform
    X_train_encoded = encoder.fit_transform(X_train_numpy, y_train_numpy)
    X_valid_encoded = encoder.transform(X_valid_numpy)
    
    # Create Polars DataFrames from the encoded results
    encoded_train_df = pl.DataFrame(
        {col: X_train_encoded[:, i] for i, col in enumerate(columns)}
    )
    encoded_valid_df = pl.DataFrame(
        {col: X_valid_encoded[:, i] for i, col in enumerate(columns)}
    )
    
    # Drop the original columns and add the encoded ones
    X_train = X_train.drop(columns)
    X_valid = X_valid.drop(columns)
    
    # Horizontal stack with encoded columns
    X_train = pl.concat([X_train, encoded_train_df], how="horizontal")
    X_valid = pl.concat([X_valid, encoded_valid_df], how="horizontal")
    
    return X_train, X_valid, encoder

def apply_target_encoding(X_test, encoder, columns):
    X_test_numpy = X_test.select(columns).to_numpy()
    X_test_encoded = encoder.transform(X_test_numpy)
    
    encoded_test_df = pl.DataFrame(
        {col: X_test_encoded[:, i] for i, col in enumerate(columns)}
    )
    X_test = X_test.drop(columns)
    X_test = pl.concat([X_test, encoded_test_df], how="horizontal")
    
    return X_test
    

# Initialize wandb
user_secrets = UserSecretsClient()
WANDB_API_KEY = user_secrets.get_secret("wandb_api")
wandb.login(key=WANDB_API_KEY)
wandb_run = wandb.init(project="playground-series-s5e4", config=asdict(cfg))

# Run the KFold training process
test_preds = get_dfs_with_kfold(cfg)

# Log final results
wandb.log({"final_predictions": wandb.Histogram(test_preds)})
wandb.finish()
gc.collect()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: masaishi to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250421_113621-la437m0o
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run swept-deluge-372
wandb: ⭐️ View project at https://wandb.ai/masaishi/playground-series-s5e4
wandb: 🚀 View run at https://wandb.ai/masaishi/playground-series-s5e4/runs/la437m0o


Training fold 1/5


/tmp/ipykernel_19/373779108.py:128: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  df = df.with_columns(
/tmp/ipykernel_19/373779108.py:186: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  df = df.with_columns(pl.col(col).cast(pl.Utf8).cast(pl.Categorical))



 pair_size: 2 



  0%|          | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_19/373779108.py:232: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  df = df.with_columns(concat_expr.alias(new_col_name).cast(pl.Categorical))
100%|██████████| 20/20 [00:04<00:00,  4.12it/s]


...

Memory usage: 112403.01 MB
Total number of columns: 233


100%|██████████| 4/4 [00:00<00:00, 19.77it/s]


Memory usage: 115856.27 MB
Total number of columns: 237
Fold 5 validation score: 12.062151672535624


wandb:                                                                                
wandb: 
wandb: Run history:
wandb: fold_1_val_score ▁
wandb: fold_2_val_score ▁
wandb: fold_3_val_score ▁
wandb: fold_4_val_score ▁
wandb: fold_5_val_score ▁
wandb:    training/rmse █▃▂▂▂▂▂▂▁▁▁▁▁▁
wandb:     valid_1/rmse █▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb: fold_1_val_score 12.09727
wandb: fold_2_val_score 12.11494
wandb: fold_3_val_score 12.1334
wandb: fold_4_val_score 12.08351
wandb: fold_5_val_score 12.06215
wandb:    training/rmse 6.13995
wandb:     valid_1/rmse 12.10138
wandb: 
wandb: 🚀 View run swept-deluge-372 at: https://wandb.ai/masaishi/playground-series-s5e4/runs/la437m0o
wandb: ⭐️ View project at: https://wandb.ai/masaishi/playground-series-s5e4
wandb: Synced 5 W&B file(s), 132 media file(s), 264 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250421_113621-la437m0o/logs


0

In [5]:
from IPython.display import display
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

df_subm = pd.read_csv(cfg.sub_path)
df_subm["Listening_Time_minutes"] = test_preds / cfg.num_fold
df_subm.to_csv('raw_submission.csv', index=False)
df_subm

,id,Listening_Time_minutes
0,750000,53.905358
1,750001,22.425277
2,750002,48.466611
3,750003,78.328807
4,750004,52.016963
...,...,...
249995,999995,10.687201
249996,999996,55.222936
249997,999997,6.158897
249998,999998,75.895708


In [6]:
df_train = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv')

df_pltpd = pd.read_csv('/kaggle/input/podcast-listening-time-prediction-dataset/podcast_dataset.csv')
df_pltpd = df_pltpd.dropna(subset=["Listening_Time_minutes"])
df_pltpd = df_pltpd.reset_index(drop=True)
df_pltpd.index = df_pltpd.index + 1000000
df_pltpd['id'] = df_pltpd.index

df_train = pd.concat([df_train, df_pltpd], axis=0)

In [7]:
df_test_with_id = df_test.copy()

df_test_with_id.loc[df_test_with_id["Number_of_Ads"] > 103.91, "Number_of_Ads"] = 0.0
df_over_ads = df_test_with_id[df_test_with_id["Number_of_Ads"] > 3]
df_over_ads["Listening_Time_minutes"] = df_over_ads["Number_of_Ads"] * 0.993
display(df_over_ads)

display(df_subm.loc[df_subm['id'].isin(df_over_ads['id'].values)])
df_subm.loc[df_subm['id'].isin(df_over_ads['id'].values), 'Listening_Time_minutes'] = df_over_ads['Listening_Time_minutes'].values
display(df_subm.loc[df_subm['id'].isin(df_over_ads['id'].values)])

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
37939,787939,Life Lessons,Episode 94,89.84,Lifestyle,60.79,Sunday,Afternoon,62.13,89.12,Neutral,88.49616


,id,Listening_Time_minutes
37939,787939,62.470018


,id,Listening_Time_minutes
37939,787939,88.49616


In [8]:
cols_to_compare = ['Episode_Title', 'Host_Popularity_percentage', 'Guest_Popularity_percentage']

df_test_with_id = df_test.copy()
df_test_with_id = df_test_with_id.dropna(subset=['Guest_Popularity_percentage'])

leaked_rows = df_test_with_id.merge(
    df_train[cols_to_compare + ['Listening_Time_minutes']].drop_duplicates(),
    on=cols_to_compare,
    how='inner'
)

mean_values = leaked_rows.groupby('id')['Listening_Time_minutes'].mean().reset_index()

display(df_subm.loc[df_subm['id'].isin(mean_values['id'].values)])
df_subm.loc[df_subm['id'].isin(mean_values['id'].values), "Listening_Time_minutes"] = mean_values["Listening_Time_minutes"].values
display(df_subm.loc[df_subm['id'].isin(mean_values['id'].values)])

,id,Listening_Time_minutes
51,750051,18.481845
52,750052,60.222327
227,750227,63.558446
243,750243,23.547206
312,750312,46.616548
...,...,...
249802,999802,38.425203
249832,999832,85.600043
249841,999841,19.826841
249854,999854,69.605168


,id,Listening_Time_minutes
51,750051,18.060370
52,750052,65.789562
227,750227,58.393200
243,750243,23.830000
312,750312,26.825190
...,...,...
249802,999802,45.055790
249832,999832,89.819210
249841,999841,17.234691
249854,999854,67.816132


In [9]:
df_subm.to_csv('submission.csv', index=False)
pl.read_csv('submission.csv')

id,Listening_Time_minutes
i64,f64
750000,53.905358
750001,22.425277
750002,48.466611
750003,78.328807
750004,52.016963
…,…
999995,10.687201
999996,55.222936
999997,6.158897
